# Test the Trend Index

## Check if index is up!

In [23]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [7]:
# es.indices.delete(index='trend', ignore=[400, 404])

{u'acknowledged': True}

In [46]:
# es.delete_by_query(index='trend', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

{u'batches': 278,
 u'deleted': 277519,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 3802,
 u'total': 277519,
 u'version_conflicts': 0}

#### Count elements in index

In [3]:
es.count(index='trend', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 1321432}

In [4]:
# counting tweets per day!
results_agg = es.search(index='trend', doc_type='tweet', size=0, body={                 
        "aggs": {
            "days" : {
            "terms" : 
                {"field" : "created_at_day", 
                "size": 1000000 }
                    }
              }
        }) 

In [5]:
# viewing lists per day
list_raw = results_agg["aggregations"]["days"]["buckets"]
new_list = sorted(list_raw, key=lambda k: k['key']) 
suma = sum(item['doc_count'] for item in new_list)
print "total", suma
for elem in new_list:
    print elem["key"], elem["doc_count"]

total 1321435
20170310 5048
20170311 8762
20170312 8135
20170313 7543
20170314 9093
20170315 8809
20170316 8483
20170317 9284
20170318 8735
20170319 8585
20170320 8491
20170321 9415
20170322 9272
20170323 8577
20170324 11727
20170325 9498
20170326 8867
20170327 8317
20170328 8194
20170329 8300
20170330 8507
20170331 9898
20170401 8673
20170402 8826
20170403 10043
20170404 11714
20170405 11804
20170406 12225
20170407 11118
20170408 9567
20170409 8500
20170410 7913
20170411 8803
20170412 9508
20170413 9029
20170414 9481
20170415 9300
20170416 7784
20170417 7903
20170418 9225
20170419 8340
20170420 8568
20170421 9568
20170422 9113
20170423 8195
20170424 7574
20170425 8299
20170426 8438
20170427 8443
20170428 9138
20170429 8593
20170430 8265
20170501 8161
20170502 8539
20170503 8824
20170504 8906
20170505 10205
20170506 9749
20170507 8355
20170508 8450
20170509 10320
20170510 8325
20170511 6224
20170512 9747
20170513 10034
20170514 7922
20170515 7775
20170516 8149
20170517 8239
20170518 80

In [ ]:
# 1. numero tweet classificati con testo
# 2. numero tweet classificati con immagine per le tre soglie di confidenza
# 3. accuratezza media classificazione immagini 
# in base alla distribuzione di frequenza delle classi

In [22]:
# counting tweets per category!
results_agg = es.search(index='trend', doc_type='tweet', size=0, body={                 
        "aggs": {
            "imgCats" : {
            "terms" : 
                {"field" : "img_categories.keyword", 
                "size": 1000000 }
                    }
              }
        }) 

RequestError: TransportError(400, u'illegal_argument_exception', u'[cardinality] unknown field [size], parser not found')

In [21]:
print results_agg

{u'hits': {u'hits': [], u'total': 1325550, u'max_score': 0.0}, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}, u'took': 7, u'aggregations': {u'imgCats': {u'buckets': [], u'sum_other_doc_count': 0, u'doc_count_error_upper_bound': 0}}, u'timed_out': False}


In [17]:
# viewing lists per day
print len(results_agg)
list_raw = results_agg["aggregations"]["imgCats"]["buckets"][:20]
for elem in list_raw:
    print elem["key"], elem["doc_count"]

5


#### GET all docs from index

In [29]:
res = es.search(index="trend", doc_type='tweet', size=10, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:20]:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840200729801416704', u'_source': {u'username': u'maslarovira', u'lang': u'und', u'text_categories': [u'hummus'], u'city': None, u'text': u'#hummus #bacall\xe0esqueixat #taronjasanguina #espinacs #yummy #brutal #espinelves #viladrau\u2026 https://t.co/bSnFsBlnyt', u'img_categories': None, u'hashtags': [u'#hummus', u'#bacall\xe0esqueixat', u'#taronjasanguina', u'#espinacs', u'#yummy', u'#brutal', u'#espinelves', u'#viladrau\u2026'], u'img_flag': True, u'timestamp_ms': u'1489154438816', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[2.392246, 41.849277], [2.392246, 41.896344], [2.454142, 41.896344], [2.454142, 41.849277]]]}, u'coords': [2.41838, 41.86547], u'id_str': u'840200729801416704', u'country': u'spain', u'id': 840200729801416704, u'media_url': None, u'created_at_day': 20170310, u'created_at_datetime': u'2017-03-10T14:00:38', u'created_at_month': 20170301}, u'_index': u'trend'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u

#### GET specific tweets

In [ ]:
es.get(index='trend', doc_type='tweet', id=880404048834416644)

In [69]:
results = es.search(index='trend', doc_type='tweet', size=100000, body={
  'query': {
    'match': {
#       'text_categories': 'hot_dog',
      'created_at_day': 20170628,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

# print results
print len(results["hits"]["hits"])

for r in results['hits']['hits'][:20]:
    print r
    print 
    
### we have 3 cases! when img_cat is {None, [], [list of categories with core and labe]}
### I believe the first case happens when we reindexed and used the classifier 
# but there was no image
### the second case was when we did batch processing and there was no img category,
# so we put an empty list. this record was not updated with the reindexing ... don't know
# why

11168
{u'_score': 1.0, u'_type': u'tweet', u'_id': u'879852557551755264', u'_source': {u'username': u'glavportal', u'lang': u'en', u'text_categories': [], u'city': u'minneapolis', u'text': u'Get the recipe Grilled Hawaiian Barbecue Chicken Best to Eat!...\n#Dinner\nhttps://t.co/GsRrRotqrk https://t.co/U6xtAODcd2', u'img_categories': [{u'score': 6.0, u'label': u'paella'}], u'hashtags': [u'#dinner'], u'img_flag': True, u'timestamp_ms': u'1498608171247', u'bounding_box': None, u'coords': None, u'id_str': u'879852557551755264', u'country': u'united states', u'id': 879852557551755264, u'media_url': u'http://pbs.twimg.com/media/DDXdPNMXsAExIDg.jpg', u'created_at_day': 20170628, u'created_at_datetime': u'2017-06-28T00:02:51', u'created_at_month': 20170601}, u'_index': u'trend'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'879852211014168576', u'_source': {u'username': u'Monaca6', u'lang': u'en', u'text_categories': [u'risotto'], u'city': None, u'text': u'RT @ericgarland: Roger Stone: Lawye

#### TEST image classifier

In [65]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
import json
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"][:10]:
    img_url = tweet["_source"]["media_url"]
    if img_url:
        res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.items
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

I have an unnatural love for a #hotdog .
@focalmark #liveauthentic #foodbeast #eeeeeats #eatfamous #feedfeed #dail…… https://t.co/JxB2QIdi6V
None
<built-in method items of dict object at 0x7f990ca5ca28>


AttributeError: 'dict' object has no attribute 'text'

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data

### Index from file

In [ ]:
import json
import os
import sys
import time
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from elasticsearch import Elasticsearch, RequestError
from processing.preprocess_tweet import process_tweet
from processing.twitter.Tweet import Tweet

In [ ]:
inputFile="/home/foodmap/data/food-tweets/food-tweets-2017-03-10.json.gz"
indexName="trend"

tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numIndex = 0
for tweet in tweetsAsDict:
    i += 1
    if i % 10 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex

    new_tweet = process_tweet(tweet, forStream=False)
    if new_tweet is None:
        continue

    try:
        if i > 1480:
            print new_tweet
        # added request_timeout to avoid elasticsearch.exceptions.ConnectionTimeout
        es.index(index=indexName, doc_type='tweet', id=new_tweet["id"], body=new_tweet)
        numIndex += 1
        # print "Indexed tweet: ", new_tweet["id"]
    except RequestError as e:
        print "Couldn't index tweet id: ", new_tweet["id"]
        print e.status_code, e.message
        time.sleep(60)

print "Processed tweets: ", i
print "Indexed tweets: ", numIndex